In [ ]:
## Module import
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from scipy import stats
import seaborn as sns
import cartopy.feature as cft
import matplotlib.path as mpath
import cmocean as cm
import string
import math
from datetime import datetime, timedelta
import proplot as pplt
import xesmf as xe
import netCDF4 as nc

In [ ]:
# open IPSL file
netCDF_TA95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_IPSL_tas_for_std_19952014.nc'
TA_CMIP95 = xr.open_mfdataset(netCDF_TA95_file)

netCDF_PR95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_IPSL_pr_for_std_19952014.nc'
PR_CMIP95 = xr.open_mfdataset(netCDF_PR95_file)

netCDF_PS95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_IPSL_psl2_for_std_19952014.nc'
PS_CMIP95 = xr.open_mfdataset(netCDF_PS95_file)

netCDF_SMB95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_IPSL_smb_for_std_19952014.nc'
SMB_CMIP95 = xr.open_mfdataset(netCDF_SMB95_file)

# open MPI file
netCDF_MPI_TA95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_tas_for_std_19952014.nc'
TA_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_TA95_file)

netCDF_MPI_PR95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_pr_for_std_19952014.nc'
PR_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_PR95_file)

netCDF_MPI_PS95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_psl2_for_std_19952014.nc'
PS_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_PS95_file)

netCDF_MPI_SMB95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_smb_for_std_19952014.nc'
SMB_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_SMB95_file)

# open UKESM file
netCDF_UKESM_TA95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_tas_for_std_19952014.nc'
TA_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_TA95_file)

netCDF_UKESM_PR95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_pr_for_std_19952014.nc'
PR_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_PR95_file)

netCDF_UKESM_PS95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_psl2_for_std_19952014.nc'
PS_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_PS95_file)

netCDF_UKESM_SMB95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_smb_for_std_19952014.nc'
SMB_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_SMB95_file)

# open topo
netCDF_topom_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/deptho_Ofx_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn.nc'
TOPOM = xr.open_mfdataset(netCDF_topom_file)

netCDF_topou_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/deptho_Ofx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc'
TOPOU = xr.open_mfdataset(netCDF_topou_file)


# Mask
netCDF_ess_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/RCM_ice_regrid_04000m.nc2'
MASK = xr.open_mfdataset(netCDF_ess_file)

SMB_UKESM_CMIP95['maskAIS'] = SMB_UKESM_CMIP95['lon'] - SMB_UKESM_CMIP95['lon'] + MASK['AIS'].values
SMB_UKESM_CMIP95['maskGRD'] = SMB_UKESM_CMIP95['lon'] - SMB_UKESM_CMIP95['lon'] + MASK['GROUND'].values

In [ ]:
sns.set_context('paper')

land_50m = cft.NaturalEarthFeature('physical', 'land', '50m', edgecolor='salmon', facecolor='papayawhip', linewidth=0.5)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
levels = np.arange(0,71,5)

fig, axs = plt.subplots(nrows = 3, ncols = 3, subplot_kw={'projection': ccrs.SouthPolarStereo()}, figsize = (10, 9), constrained_layout=True)
axs = axs.flatten()


for n, ax in enumerate(axs):
    ax.text(0.025, 0.93, string.ascii_lowercase[n]+')', transform=ax.transAxes, color='black', fontweight="bold")
    ax.set_extent([-180, 180, -90, -60], crs=ccrs.PlateCarree())
    ax.coastlines(resolution='50m', color='black',)
    gl0 = ax.gridlines(draw_labels=False, linewidth=0.75, color='gray', alpha=0.7, linestyle='--', dms=True, x_inline=False, y_inline=False)
    gl0.top_labels = False
    gl0.right_labels = False
    gl0.xlocator = mticker.FixedLocator([-180, -135, -90, -45, 0, 45, 90, 135, 180])
    gl0.ylocator = mticker.FixedLocator([-90, -80, -70, -60])
    gl0.xformatter = LongitudeFormatter()
    gl0.yformatter = LatitudeFormatter()
    gl0.xlabel_style = {'size': 12, 'color': 'gray'}
    gl0.ylabel_style = {'size': 12, 'color': 'gray'}   
    #ax.set_boundary(circle, transform=ax.transAxes)
    
fmt = {}
strs = ['0', '5','10', '15', '20', '25', '30', '35', '40', '45', '50', '55', '60']
al = np.array([ 0., 5., 10., 15., 20., 25., 30., 35., 40., 45., 50., 55.,60.,65.,70.])
for l, s in zip(al, strs):
    fmt[l] = s

    
plt.figtext(0.01, 0.805, 'Across member standard', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.03, 0.805, 'deviation of the 1995-2014', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.05, 0.805, 'mean SMB', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.01, 0.485, 'Across member standard', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.03, 0.485, 'deviation of the 1995-2014', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.05, 0.485, 'mean air temperature', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.25, 0.97, 'MPI', fontweight="bold")
plt.figtext(0.538, 0.97, 'IPSL', fontweight="bold")
plt.figtext(0.812, 0.97, 'UKESM', fontweight="bold")
plt.figtext(0.01, 0.17, 'Across member standard', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.03, 0.17, 'deviation of the 1995-2014', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.05, 0.17, 'mean precipitation', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")


# smb 1995-2014
p6 = axs[1].pcolormesh(SMB_CMIP95.lon, SMB_CMIP95.lat, SMB_CMIP95['smb_std'].where(SMB_UKESM_CMIP95['maskAIS']>0)*31536000,  cmap=cm.cm.haline, vmin = 0, vmax =30, transform=ccrs.PlateCarree(), shading='auto')
p7 = axs[0].pcolormesh(SMB_MPI_CMIP95.lon, SMB_MPI_CMIP95.lat, SMB_MPI_CMIP95['smb_std'].where(SMB_UKESM_CMIP95['maskAIS']>0)*31536000,  cmap=cm.cm.haline, vmin = 0, vmax =30,transform=ccrs.PlateCarree(), shading='auto')
p8 = axs[2].pcolormesh(SMB_UKESM_CMIP95.lon, SMB_UKESM_CMIP95.lat, SMB_UKESM_CMIP95['smb_std'].where(SMB_UKESM_CMIP95['maskAIS']>0)*31536000,  cmap=cm.cm.haline, vmin = 0, vmax =30,transform=ccrs.PlateCarree(), shading='auto')

# Air temperature 1995-2014
p0 = axs[4].pcolormesh(TA_CMIP95.lon, TA_CMIP95.lat, TA_CMIP95['tas_std'], cmap=cm.cm.thermal, vmin = 0, vmax =1, transform=ccrs.PlateCarree(), shading='auto')
p1 = axs[3].pcolormesh(TA_MPI_CMIP95.lon, TA_MPI_CMIP95.lat, TA_MPI_CMIP95['tas_std'], cmap=cm.cm.thermal, vmin = 0, vmax =1, transform=ccrs.PlateCarree(), shading='auto')
p2 = axs[5].pcolormesh(TA_UKESM_CMIP95.lon, TA_UKESM_CMIP95.lat, TA_UKESM_CMIP95['tas_std'], cmap=cm.cm.thermal, vmin = 0, vmax =1, transform=ccrs.PlateCarree(), shading='auto')
axs[3].text(0.025, 0.93, 'd)', transform=axs[3].transAxes, color='white', fontweight="bold")
axs[4].text(0.025, 0.93, 'e)', transform=axs[4].transAxes, color='white', fontweight="bold")
axs[5].text(0.025, 0.93, 'f)', transform=axs[5].transAxes, color='white', fontweight="bold")

# precipitation 1995-2014 + contour pressure
p3 = axs[7].pcolormesh(PR_CMIP95.lon, PR_CMIP95.lat, PR_CMIP95['pr_std']*31536000, vmin = 0, vmax =80, cmap=cm.cm.rain,  transform=ccrs.PlateCarree(), shading='auto')
p33 = axs[7].contour(PS_CMIP95.lon, PS_CMIP95.lat, PS_CMIP95['psl_std'], levels, colors='crimson', linewidths = 1, transform=ccrs.PlateCarree())
axs[7].clabel(p33, p33.levels, inline=True, fmt=fmt, fontsize=5)

p4 = axs[6].pcolormesh(PR_MPI_CMIP95.lon, PR_MPI_CMIP95.lat, PR_MPI_CMIP95['pr_std']*31536000, cmap=cm.cm.rain, vmin = 0, vmax =80, transform=ccrs.PlateCarree(), shading='auto')
p44 = axs[6].contour(PS_MPI_CMIP95.lon, PS_MPI_CMIP95.lat, PS_MPI_CMIP95['psl_std'], levels, colors='crimson', linewidths = 1, transform=ccrs.PlateCarree())
axs[6].clabel(p44, p44.levels, inline=True, fmt=fmt, fontsize=5)

p5 = axs[8].pcolormesh(PR_UKESM_CMIP95.lon, PR_UKESM_CMIP95.lat, PR_UKESM_CMIP95['pr_std']*31536000, cmap=cm.cm.rain, vmin = 0, vmax =80, transform=ccrs.PlateCarree(), shading='auto')
p55 = axs[8].contour(PS_UKESM_CMIP95.lon, PS_UKESM_CMIP95.lat, PS_UKESM_CMIP95['psl_std'], levels, colors='crimson', linewidths = 1, transform=ccrs.PlateCarree())
axs[8].clabel(p55, p55.levels, fmt=fmt, fontsize=5)


ax_cb = plt.axes([0.069, 0.705, 0.007, 0.2 ])
cb = plt.colorbar(p8, cax=ax_cb, orientation='vertical')
cb.ax.set_ylabel('Standard deviation (kg m$^{2}$ yr$^{-1}$)');

ax_cb1 = plt.axes([0.069, 0.385, 0.007, 0.2 ])
cb1 = plt.colorbar(p2, cax=ax_cb1, orientation='vertical')
cb1.ax.set_ylabel('standard deviation (°C)');

ax_cb2 = plt.axes([0.069, 0.07, 0.007, 0.2 ])
cb2 = plt.colorbar(p5, cax=ax_cb2, orientation='vertical')
cb2.ax.set_ylabel('Standard deviation (kg m$^{2}$ yr$^{-1}$)');
       
plt.tight_layout(rect=[0.10, 0, 1, 0.97])
plt.savefig('/Users/jcaillet/Documents/Elmer/CMIP6/FIGURES/atmo_std.png',dpi=200)